In [1]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import util
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")


from dotenv import load_dotenv
import os

env_path = '../../data/.env'
load_dotenv(dotenv_path=env_path)

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
AQI_API_KEY = os.getenv("AQI_API_KEY")

In [2]:
csv_file="../../data/jing'an-monitoring station-air-quality.csv"
util.check_file_path(csv_file)

File successfully found at the path: ../../data/jing'an-monitoring station-air-quality.csv


In [3]:
country="china"
city = "shanghai"
street = "jinganjiancezhan"
aqicn_url="https://api.waqi.info/feed/@486"

latitude, longitude = util.get_city_coordinates(city)
today = datetime.date.today()
latitude, longitude, today

(31.23, 121.47, datetime.date(2024, 11, 12))

In [4]:
project = hopsworks.login(project='AMomozZz')

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1163416


In [5]:
secrets = util.secrets_api(project.name)
try:
    secrets.create_secret("AQI_API_KEY", AQI_API_KEY)
except hopsworks.RestAPIError:
    AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value

Connected. Call `.close()` to terminate connection gracefully.


In [6]:
try:
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
except hopsworks.RestAPIError:
    print("It looks like the AQI_API_KEY doesn't work for your sensor. Is the API key correct? Is the sensor URL correct?")

aq_today_df.head()

,pm25,country,city,street,date,url
0,132.0,china,shanghai,jinganjiancezhan,2024-11-12,https://api.waqi.info/feed/@486


In [7]:
df = pd.read_csv(csv_file,  parse_dates=['date'], skipinitialspace=True)
df

,date,pm25,pm10,o3,no2,so2,co
0,2024-11-01,20.0,45.0,44.0,13.0,3.0,5.0
1,2024-11-02,103.0,58.0,47.0,19.0,3.0,6.0
2,2024-11-03,140.0,48.0,58.0,18.0,4.0,6.0
3,2024-11-04,124.0,21.0,31.0,10.0,3.0,3.0
4,2024-11-05,45.0,20.0,35.0,12.0,3.0,3.0
...,...,...,...,...,...,...,...
3954,2018-12-31,NaN,33.0,27.0,10.0,4.0,5.0
3955,2017-09-10,NaN,31.0,28.0,18.0,4.0,11.0
3956,2016-01-03,NaN,95.0,30.0,35.0,11.0,10.0
3957,2014-12-31,NaN,61.0,21.0,21.0,16.0,6.0


In [8]:
# These commands will succeed if your CSV file didn't have a `median` or `timestamp` column
df = df.rename(columns={"median": "pm25"})
df = df.rename(columns={"timestamp": "date"})

df_aq = df[['date', 'pm25']]
df_aq['pm25'] = df_aq['pm25'].astype('float32')
df_aq

,date,pm25
0,2024-11-01,20.0
1,2024-11-02,103.0
2,2024-11-03,140.0
3,2024-11-04,124.0
4,2024-11-05,45.0
...,...,...
3954,2018-12-31,NaN
3955,2017-09-10,NaN
3956,2016-01-03,NaN
3957,2014-12-31,NaN


In [9]:
# Cast the pm25 column to be a float32 data type
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3959 entries, 0 to 3958
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3959 non-null   datetime64[ns]
 1   pm25    3952 non-null   float32       
dtypes: datetime64[ns](1), float32(1)
memory usage: 46.5 KB


In [10]:
df_aq.dropna(inplace=True)
df_aq

,date,pm25
0,2024-11-01,20.0
1,2024-11-02,103.0
2,2024-11-03,140.0
3,2024-11-04,124.0
4,2024-11-05,45.0
...,...,...
3947,2014-03-27,108.0
3948,2014-03-28,103.0
3949,2014-03-29,153.0
3950,2014-03-30,173.0


In [14]:
# Your sensor may have columns we won't use, so only keep the date and pm25 columns
# If the column names in your DataFrame are different, rename your columns to `date` and `pm25`
df_aq['country']=country
df_aq['city']=city
df_aq['street']=street
df_aq['url']=aqicn_url

df_aq.sort_values(by='date', ascending=True).tail(20)

,date,pm25,country,city,street,url
31,2024-10-22,104.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
32,2024-10-23,95.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
33,2024-10-24,46.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
34,2024-10-25,47.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
35,2024-10-26,23.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
36,2024-10-27,34.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
37,2024-10-28,115.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
38,2024-10-29,49.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
39,2024-10-30,54.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
40,2024-10-31,49.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486


In [18]:
earliest_aq_date = pd.Series.min(df_aq['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

weather_df = util.get_historical_weather(city, earliest_aq_date, str(today), latitude, longitude)

hourly_df = util.get_yesterday_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city

weather_df = pd.concat([weather_df, daily_df.head(1)])

for i in range(1, 21):
    df_temp = df_aq.copy()
    df_temp['date'] = pd.to_datetime(df_temp['date']) + pd.Timedelta(days=i)
    weather_df = pd.merge(weather_df, df_temp[['date','pm25']], how='left', on="date")
    weather_df = weather_df.rename(columns={"pm25": f"pm25_{i}"})

weather_df.dropna(inplace=True)

# df_temp = df_aq.copy()
# df_temp['date'] = pd.to_datetime(df_temp['date'])
# df_temp['day_month'] = df_temp['date'].dt.strftime('%m-%d')
# average_pm25 = df_temp.groupby('day_month')['pm25'].mean().round(2)
# df_temp = df_temp.merge(average_pm25, on='day_month', suffixes=('', '_avg'))
# weather_df = pd.merge(weather_df, df_temp[['date','pm25_avg']], how='left', on="date")

weather_df.sort_values(by='date', ascending=True).tail(10)

Coordinates 31.247802734375°N 121.5°E
Elevation 3.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_1,pm25_2,pm25_3,pm25_4,...,pm25_11,pm25_12,pm25_13,pm25_14,pm25_15,pm25_16,pm25_17,pm25_18,pm25_19,pm25_20
3958,2024-11-02,17.463247,0.0,24.215185,335.282562,shanghai,20.0,49.0,54.0,49.0,...,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0,60.0,43.0
3959,2024-11-03,17.740332,0.0,8.149847,16.130766,shanghai,103.0,20.0,49.0,54.0,...,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0,60.0
3960,2024-11-04,17.409084,0.0,19.645580,10.283305,shanghai,140.0,103.0,20.0,49.0,...,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0
3961,2024-11-05,13.817414,0.0,19.852295,358.205353,shanghai,124.0,140.0,103.0,20.0,...,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0
3962,2024-11-06,13.725750,0.0,17.589418,15.969136,shanghai,45.0,124.0,140.0,103.0,...,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0
3963,2024-11-07,14.234082,0.0,14.450535,34.656887,shanghai,35.0,45.0,124.0,140.0,...,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0
3964,2024-11-08,17.711164,0.0,18.275984,44.213470,shanghai,35.0,35.0,45.0,124.0,...,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0
3965,2024-11-09,18.850750,0.5,21.862406,43.717545,shanghai,55.0,35.0,35.0,45.0,...,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0
3966,2024-11-10,16.086168,1.2,18.553974,0.802675,shanghai,46.0,55.0,35.0,35.0,...,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0
3967,2024-11-11,15.100000,0.0,8.161764,41.423569,shanghai,56.0,46.0,55.0,35.0,...,49.0,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0


In [19]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3819 entries, 20 to 3967
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         3819 non-null   datetime64[ns]
 1   temperature_2m_mean          3819 non-null   float32       
 2   precipitation_sum            3819 non-null   float32       
 3   wind_speed_10m_max           3819 non-null   float32       
 4   wind_direction_10m_dominant  3819 non-null   float32       
 5   city                         3819 non-null   object        
 6   pm25_1                       3819 non-null   float32       
 7   pm25_2                       3819 non-null   float32       
 8   pm25_3                       3819 non-null   float32       
 9   pm25_4                       3819 non-null   float32       
 10  pm25_5                       3819 non-null   float32       
 11  pm25_6                       3819 non-null   fl

In [20]:
import great_expectations as ge
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

In [22]:
import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")
for i in range(1, 21):
    expect_greater_than_zero(f"pm25_{i}")

In [23]:
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.


In [24]:
dict_obj = {
    "country": country,
    "city": city,
    "street": street,
    "aqicn_url": aqicn_url,
    "latitude": latitude,
    "longitude": longitude
}

# Convert the dictionary to a JSON string
str_dict = json.dumps(dict_obj)

try:
    secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)
except hopsworks.RestAPIError:
    print("SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("SENSOR_LOCATION_JSON").value
    print(f"{existing_key}")

SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.
{"country": "china", "city": "shanghai", "street": "jinganjiancezhan", "aqicn_url": "https://api.waqi.info/feed/@486", "latitude": 31.23, "longitude": 121.47}


In [25]:
air_quality_fg = fs.get_or_create_feature_group(
    name='new_aq',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

In [26]:
_ = air_quality_fg.insert(df_aq)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1345972
2024-11-12 18:59:34,247 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1345972


Uploading Dataframe: 0.00% |          | Rows 0/3952 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: new_aq_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163416/jobs/named/new_aq_1_offline_fg_materialization/executions


In [27]:
air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("country", "Country where the air quality was measured (sometimes a city in acqcn.org)")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("street", "Street in the city where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")

In [28]:
# Get or create feature group 
weather_fg = fs.get_or_create_feature_group(
    name='new_w',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city', 'date'],
    event_time="date",
    expectation_suite=weather_expectation_suite
) 

In [29]:
# Insert data
_ = weather_fg.insert(weather_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1345973
2024-11-12 18:59:58,289 INFO: 	22 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1163416/fs/1154119/fg/1345973


Uploading Dataframe: 0.00% |          | Rows 0/3819 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: new_w_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1163416/jobs/named/new_w_1_offline_fg_materialization/executions


In [30]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")
for i in range(1, 21):
    weather_fg.update_feature_description(f"pm25_{i}", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")

In [31]:
display(air_quality_fg.read().sort_values(by=['date'], ascending=True).tail(10))
display(weather_fg.read().sort_values(by=['date'], ascending=True).tail(10))

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.46s) 


,date,pm25,country,city,street,url
3791,2024-11-01 00:00:00+00:00,20.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3722,2024-11-02 00:00:00+00:00,103.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
708,2024-11-03 00:00:00+00:00,140.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
540,2024-11-04 00:00:00+00:00,124.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
348,2024-11-05 00:00:00+00:00,45.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1286,2024-11-06 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
1370,2024-11-07 00:00:00+00:00,35.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3121,2024-11-08 00:00:00+00:00,55.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
2555,2024-11-09 00:00:00+00:00,46.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486
3039,2024-11-10 00:00:00+00:00,56.0,china,shanghai,jinganjiancezhan,https://api.waqi.info/feed/@486


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.86s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,pm25_1,pm25_2,pm25_3,pm25_4,...,pm25_11,pm25_12,pm25_13,pm25_14,pm25_15,pm25_16,pm25_17,pm25_18,pm25_19,pm25_20
422,2024-11-02 00:00:00+00:00,17.463247,0.0,24.215185,335.282562,shanghai,20.0,49.0,54.0,49.0,...,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0,60.0,43.0
2250,2024-11-03 00:00:00+00:00,17.740332,0.0,8.149847,16.130766,shanghai,103.0,20.0,49.0,54.0,...,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0,60.0
3676,2024-11-04 00:00:00+00:00,17.409084,0.0,19.645580,10.283305,shanghai,140.0,103.0,20.0,49.0,...,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0,108.0
2825,2024-11-05 00:00:00+00:00,13.817414,0.0,19.852295,358.205353,shanghai,124.0,140.0,103.0,20.0,...,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0,71.0
1213,2024-11-06 00:00:00+00:00,13.725750,0.0,17.589418,15.969136,shanghai,45.0,124.0,140.0,103.0,...,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0,68.0
2444,2024-11-07 00:00:00+00:00,14.234082,0.0,14.450535,34.656887,shanghai,35.0,45.0,124.0,140.0,...,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0,73.0
2238,2024-11-08 00:00:00+00:00,17.711164,0.0,18.275984,44.213470,shanghai,35.0,35.0,45.0,124.0,...,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0,52.0
1079,2024-11-09 00:00:00+00:00,18.850750,0.5,21.862406,43.717545,shanghai,55.0,35.0,35.0,45.0,...,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0,48.0
957,2024-11-10 00:00:00+00:00,16.086168,1.2,18.553974,0.802675,shanghai,46.0,55.0,35.0,35.0,...,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0,82.0
3170,2024-11-11 00:00:00+00:00,15.100000,0.0,8.161764,41.423569,shanghai,56.0,46.0,55.0,35.0,...,49.0,54.0,49.0,115.0,34.0,23.0,47.0,46.0,95.0,104.0
